In [75]:
from langchain_groq import ChatGroq
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

In [76]:
# Membuat model
model = ChatGroq(model="llama3-8b-8192")

In [77]:
# Membuat prompr template
starting_prompt = ChatPromptTemplate.from_messages([
    ("system", "Kamu adalah seorang analis sentimen."),
    ("human", "Perhatikan ulasan ini: {ulasan}.\nDari ulasan ini, pisahkan ulasan yang positif dan negatif.")
])

In [78]:
# Fungsi Prompt Branches
def get_positif_prompt(ulasan):
    positif_prompt = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang analis sentimen."),
        ("human", "Dari ulasan ini: {ulasan}, temukan semua aspek yang positif dan jelaskan lebih jelas.")
    ])
    
    return positif_prompt.format_prompt(ulasan=ulasan).to_string()

def get_negatif_prompt(ulasan):
    negatif_prompt = ChatPromptTemplate.from_messages([
        ("system", "Kamu adalah seorang analis sentimen."),
        ("human", "Dari ulasan ini: {ulasan}, temukan semua aspek yang negatif dan jelaskan lebih jelas.")
    ])
    
    return negatif_prompt.format_prompt(ulasan=ulasan).to_string()

# Fungsi gabung sentimen
def gabung_sentimen(positif, negatif):
    return f"Sentimen Positif:\n{positif}\n\nSentimen Negatif:\n{negatif}"

In [79]:
# Membuat Branch
positif_branch = RunnableLambda(lambda x:
    # get_positif_prompt(x) | model | StrOutputParser()                 # ❌
    # model.invoke(get_positif_prompt(x)).content | StrOutputParser()   # ❌
    StrOutputParser().parse(model.invoke(get_positif_prompt(x)))        # ✅ 😭
)

negatif_branch = RunnableLambda(lambda x: 
    # get_negatif_prompt(x) | model | StrOutputParser()                 # ❌
    # model.invoke(get_negatif_prompt(x)).content | StrOutputParser()   # ❌
    StrOutputParser().parse(model.invoke(get_negatif_prompt(x)))        # ✅ 😭
)

In [86]:
# Membuat chain
sentimen_chain = (
    starting_prompt
    | model
    | StrOutputParser()
    | RunnableParallel(branches={
        "positif": positif_branch,
        "negatif": negatif_branch
    })
    | RunnableLambda(lambda x: gabung_sentimen(x["branches"]["positif"], x["branches"]["negatif"]))
)

In [87]:
# Eksekusi
ulasan = """Produk MackBook ini puya banyak fitur yang sangat berkualitas dengan banyak keunggulannya
Tetapi karena jumlah portnya yang sedikit membuat aku tidak suka. Ini membuat produk ini sangat tidak sesuai dengan harga mahasiswa. Dasar, merek cari uang.
"""

result = sentimen_chain.invoke({"ulasan": ulasan})
print(result)

Sentimen Positif:
content='As a sentiment analyst, I\'ve analyzed the given reviews and identified the following aspects that are positively mentioned about the MacBook product:\n\n1. **High-quality features**: The review mentions that the MacBook has "banyak fitur yang sangat berkualitas" which translates to "many high-quality features". This suggests that the product has a lot of advanced and reliable features that make it stand out.\n2. **Many advantages**: The review highlights the many advantages of the MacBook, implying that it has a lot of benefits that make it a valuable product.\n\nThese positive aspects are mentioned to emphasize the product\'s strengths and capabilities, suggesting that the reviewer is impressed with its performance and features.\n\nPlease note that the negative reviews are also provided, but as a sentiment analyst, I focused on identifying the positive aspects mentioned in the given reviews.' additional_kwargs={} response_metadata={'token_usage': {'completi

### Conclutions

CARANYA MASIH SALAH😭😭😭